In [1]:
import unsloth
from datasets import load_dataset
from transformers import AutoTokenizer, TextStreamer
from unsloth import FastLanguageModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import os
from tqdm import tqdm

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.6.0+cu118)
    Python  3.12.8 (you have 3.12.9)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
# Load test split
dataset = load_dataset("json", data_files="responses_10000.jsonl", split="train")
# dataset = dataset[-1000:]

In [3]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
You are a legal expert. Provide accurate, well-reasoned legal insights using proper legal terminology.
Maintain a professional, objective tone. Be specific about which laws or legal principles apply.
Explain the person's rights, cite the relevant statute(s), and give a clear legal opinion.
When unsure, advise consulting a qualified attorney.

### Input:
{}

### Response:
{}"""

# Define the end-of-sequence token
EOS_TOKEN = '<end>'

In [4]:
def formatting_prompts_func(examples):
    return {
        'text': [
            prompt_style.format(q, a) + EOS_TOKEN
            for q, a in zip(examples['question'], examples['response'])
        ]
    }

# Apply the function to format the dataset
dataset_finetune = dataset.map(formatting_prompts_func)
dataset_finetune = dataset_finetune.select(range(len(dataset_finetune) - 1000, len(dataset_finetune)))

# Print the formatted text from the first example to verify
# print("Formatted prompt example:")
# print(dataset_finetune["text"][0])

# dataset_finetune = dataset_finetune[-1000:]

In [5]:
# Load fine-tuned model from Unsloth
device = "cuda" if torch.cuda.is_available() else "cpu"

model_path = "llama3_gguf"  # Folder containing config.json, pytorch_model.bin, etc.

# Load the full fine-tuned model (no need to call load_adapter)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=2048,
    dtype=torch.float16,  # or torch.float16
    load_in_4bit=True,  # if you want quantization (depends on your use case)
    device_map="auto",
)

# model = model.to(device)

# Set to eval mode
# model.eval()

# Optional: Use a streamer to print generated text
# streamer = TextStreamer(tokenizer)

c:\Users\Zachary Daniels\anaconda3\envs\ml\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    NVIDIA GeForce RTX 4070. Num GPUs = 1. Max memory: 11.994 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 8.9. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
model.load_adapter("unslothllama-3-8b-Instruct")

In [ ]:
import json

results = []

# Run inference on test set
for example in tqdm(dataset_finetune, desc="Generating responses"):
    input_text = example["text"]  # Adjust field name if needed
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(model.device)

    with torch.no_grad():
        output = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=500
        )
        decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    # Store input-output pair
    results.append({
        "input": input_text,
        "output": decoded_output
    })

# Save to JSON file
with open("generated_outputs.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)


Generating responses:   0%|          | 0/1000 [00:00<?, ?it/s]